In [5]:
import requests
from lxml import etree
import simplejson
import re
import operator
from functools import reduce

In [6]:
ua = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'
headers = {
    'User-agent': ua
}

In [11]:
def get_url_html(url):
    with requests.Session() as session:
        response = session.get(url, headers=headers)
        text = response.text
        html = etree.HTML(text)
    return html

In [28]:
def get_url_json(url):
    with requests.Session() as session:
        response = session.get(url, headers=headers)
        text = response.text
        text_json = simplejson.loads(text)
    return text_json

In [30]:
def parse_lyric(text_json):
    try:
        lyric = text_json.get('lrc').get('lyric')
        regex = re.compile(r'\[.*\]')
        final_lyric = re.sub(regex, '', lyric).strip()
        return final_lyric
    except AttributeError as k:
        print(k)
        pass

In [17]:
def get_album(html):
    album_ids = html.xpath("//ul[@id='m-song-module']/li/p/a/@href")
    album_names = html.xpath("//ul[@id='m-song-module']/li/p/a/text()")
    album_ids = [ids.split('=')[-1] for ids in album_ids]
    return album_ids, album_names

In [23]:
def get_all_song_id(album_ids):
    with requests.Session() as session:
        all_song_ids, all_song_names = [], []
        for album_id in album_ids:
            one_album_url = "https://music.163.com/album?id="+str(album_id)
            response = session.get(one_album_url, headers=headers)
            text = response.text
            html = etree.HTML(text)
            album_song_ids = html.xpath("//ul[@class='f-hide']/li/a/@href")
            album_song_names = html.xpath("//ul[@class='f-hide']/li/a/text()")
            album_song_ids = [ids.split('=')[-1] for ids in album_song_ids]

            all_song_ids.append(album_song_ids)
            all_song_names.append(album_song_names)

    return all_song_ids, all_song_names

In [12]:
singer_id = '14242'
album_url = "https://music.163.com/artist/album?id="+str(singer_id)+"&limit=150&offset=0"
html_album = get_url_html(album_url)

In [18]:
album_ids, album_names = get_album(html_album)

In [24]:
all_song_ids, all_song_names = get_all_song_id(album_ids)

In [27]:
all_song_ids = reduce(operator.add, all_song_ids)
all_song_names = reduce(operator.add, all_song_names)
print(all_song_ids)
print(all_song_names)

['1458531166', '1454714164', '1439042896', '1397273856', '1397275767', '1397275768', '1397275769', '1334406036', '1334405083', '1334405081', '1334405080', '1334405085', '1334405071', '1334405082', '1334406044', '1334406051', '1334405084', '1334406052', '1334405089', '1334406050', '1334406053', '1302084434', '541480237', '541499254', '541511246', '541511247', '498555560', '498555562', '498555561', '498577482', '432821964', '432821965', '432821966', '432821967', '37988127', '37955171', '37988128', '37955172', '37988129', '37955173', '37988130', '37955174', '37988131', '37955175', '37955176', '37955177', '37988132', '37955178', '32465903', '32465904', '32465905', '32465906', '28772113', '28772114', '28772115', '28772116', '27804431', '27804432', '26429290', '26429291', '26326160', '26326161', '26326162', '26326163', '26326164', '26326165', '26326166', '26326167', '26326168', '26326169', '26326170', '26326171', '26326172', '26108073', '26108074', '26108075', '26108076', '431900', '431903',

In [32]:
for song_id, song_name in zip(all_song_ids, all_song_names):
    url_song = 'http://music.163.com/api/song/lyric?' + 'id=' + str(song_id) + '&lv=1&kv=1&tv=-1'
    json_text = get_url_json(url_song)
    print(song_name)
    try:
        with open("hoshino.txt", 'a') as f:
            f.write(parse_lyric(json_text))
    except Exception as e:
        pass

折り合い
うちで踊ろう (Potluck Mix)
うちで踊ろう
Same Thing
さらしもの
Ain't Nobody Know
私
Pop Virus
恋
Get a Feel
肌
Pair Dancer
Present
Dead Leaf
Kids
Continues
サピエンス
アイデア
Family Song
Nothing
Hello Song
アイデア
ドラえもん
ここにいないあなたへ
The Shower
ドラえもんのうた (House ver.)
Family Song
肌
プリン
KIDS (House ver.)
恋
Drinking Dance
Continues
雨音 (House ver.)
時よ
Week End
SUN
ミスユー
Soul
口づけ
地獄でなぜ悪い
Nerd Strut (Instrumental)
'NoneType' object has no attribute 'get'
桜の森
Crazy Crazy
Snow Men
Down Town
夜
Friend Ship
SUN
Moon Sick
いち に さん
マッドメン
Crazy Crazy
桜の森
Night Troop
海を掬う (House ver.)
地獄でなぜ悪い
地獄でなぜ悪い （Karaoke)
ギャグ
ダスト
化物
ワークソング
夢の外へ
フィルム
ツアー
スカート
生まれ変わり
パロディ
季节
レコードノイズ
知らない
ある车掌
Stranger
知らない
ダンサー
季节
おもかげ (House ver.)
夢の外へ
パロディ
彼方
電波塔 (House ver.)
フィルム
もしも
乱视
次は何に産まれましょうか (House ver.)
落下 (House ver.)
エピソード
汤気
変わらないまま
くだらないの中に
布団
バイト
営业
ステップ
未来
喧哗
ストーブ
日常
予想
くだらないの中に
歌を歌うときは
湯気
ブランコ (House ver.)
くせのうた (House ver.)
ばらばら
グー
キッチン
茶碗
デイジーお味噌汁 (Instrumental)
'NoneType' object has no attribute 'get'
夜中呗
老夫妇
くせのうた
兄妹
子供
さようならのうみ (Instrument

In [34]:
file = open('hoshino1.txt')
lines = file.readlines()  
type(lines)

list

In [60]:
file = open('hoshino.txt', 'r') # 要去掉空行的文件 
file1 = open('hoshino1.txt', 'w') # 生成没有空行的文件

In [61]:
for line in file.readlines():
    if line == '\n':
        line = line.strip('\n')
    line = line.replace(u'\u3000',u'')
    line = line.replace(' ', '')
    file1.write(line)
file.close()
file1.close()

In [72]:
file1 = open('hoshino1.txt', 'r') 
file2 = open('hoshino2.txt', 'w')
for line in file1.readlines():
    if line == '\n':
        line = line.strip('\n')
    line = re.sub('[a-zA-Z]','',line)
    print(line)
    file2.write(line)
file1.close()
file2.close()

君の着信に開く封筒は

踊り叱る赤い絵文字が

あの日隠し事したのばれたから

謝り方考えなきゃな

いつもの僕らを

見ればまあまあ平和なんだろう

可愛げのある危機と

そうじゃない方胸に抱いて

愛してるよ君を

探してるよいつも

他人のようで違う

2人の折り合いを

愛してるよ君を

探してるよいつも

家族のように映る

2人の折り合いを

こんな時さえも誕生日は来て

祝うだろう日々は続けと

うん、ごめんなさい 

から始まる

バカ極まる毎度のことだが

慣れてるわけじゃない

藁をつかめテレビとかね

気分変えるのに最適だ

えー

多分漏れずクソみたいな話題

観るくらいなら君と喧嘩していたい

神へアーメン君への謝罪

まだ隠れラーメン食べたのしょない

どこへ行くの僕ら

いがみ合うよ彼ら

空は晴れて風は

髪を撫でていくわ

家の中で僕ら

道の上で僕ら

空は晴れて風は

心撫でていくわ

愛してるよ君を

探してるよいつも

他人のようで違う

2人の折り合いを

愛してるよ君を

探してるよいつも

家族のように映る

2人の折り合いを

たまに重なり合うよな僕ら

扉閉じれば明日が生まれるなら

遊ぼう一緒に

うちで踊ろう

ひとり踊ろう

変わらぬ鼓動弾ませろよ

生きて踊ろう

僕らそれぞれの場所で

重なり合うよ

うちで歌おう

悲しみの向こう

全ての歌で手を繋ごう

生きてまた会おう

僕らそれぞれの場所で

重なり合えそう

うちで踊ろう

ひとり踊ろう

変わらぬ鼓動弾ませろよ

生きて踊ろう

僕らそれぞれの場所で

重なり合うよ

うちで歌おう

悲しみの向こう

全ての歌で手を繋ごう

生きてまた会おう

僕らそれぞれの場所で

重なり合えそうだ

たまに重なり合うよな僕ら

扉閉じれば明日が生まれるなら

遊ぼう一緒に

うちで踊ろう

ひとり踊ろう

変わらぬ鼓動弾ませろよ

生きて踊ろう

僕らそれぞれの場所で重なり合うよ

うちで歌おう

悲しみの向こう

全ての歌で手を繋ごう

生きてまた会おう

僕らそれぞれの場所で重なり合えそうだ

生まれて独りステージに立って

フィナーレまでは残り何公演

人差し指の隣の指は

まだ仕舞っておいて

また後世

この輝きは僕のじゃなくて

世の光映して


春に手を振る

何もない日々よ

さようならまた逢うまで

おやすみあなたよ

ドアノブを掴んだら

仕事に行こう

何度も何度も繋いだ手が

いつまでも輝けばいいな

何度も何度も見上げた背中はもう

前を向いたまま

何度も何度も繋いだ手が

いつまでも輝けばいいな

何度も何度も掴んだ背中はもう

前を向いたまま

生まれ変わりがあるのなら

人は歌なんて歌わないさ

笑い声遠くに逃げて

子供の僕はさようなら

何度も何度も過ごした日を

いつまでも憶えられるかな

何度も何度も交わした体はもう

胸に秘めたまま

木漏れ日がラムネの玉が

固く乾いた空き地の本が

叫び声遠くで逃げて

子供の僕はさようなら

何度も何度も繋いだ手が

いつまでも輝けばいいな

何度も何度も交わした言葉はもう

風に消えたまま

決してもう二度と戻らぬ日が

いつまでも輝けばいいな

何度も何度も見上げた背中はもう

前を向いたまま

前を前を向いたまま

過ぎたはずの夏は止まって

想像でつくる今窓を開ける

南風は一拍子さ

蝶々の飛び方を真似て踊る

誰も日々を騙すだろ

こんなメロディ歌いながら

ララララララララララララララ

日々のパロディ

遠い先へ僕を運ぶ

ぜんぶ嘘さ汗の混じった

妄想がつくる川海へつづく

いつか誰か拾うだろう

変なメロディ飛ばすよほら

ララララララララララララララ

日々のメロディ

遠い先の僕を作る

ダダダダダダダ......

明日へ誤摩化せ

ララララララララララララララ

日々のメロディ

先の先へ歌えば

ララララララララララララララ

日々のパロディ

遠い先へ僕を運ぶ

歌はつづく

柳が揺れあの娘の

ああ街の灯がゆれてる

平屋の角細道

湯気が狭いお空に

消えてゆく消えてゆく我は

溶けてゆく

誰かに微笑んだ

その季節思い出す

何かに取り付かれていたように

柳は揺れあの日の

ああ切なさがゆれてる

心の隅喜び

白い息がお空に

消えてゆく消えてゆく我は

落ちてゆく

誰かと微笑んだ

その季節思い出す

何かに取り付かれていたように

誰かが微笑んだ

その季節思い出して

何かに取り付かれていたように

遠く曇った

どうにもならない夜には

心の針に思い出の溝を当てよう

流れる言葉



In [74]:
file2 = open('hoshino2.txt')
lines = file2.readlines()  
print(lines)

['君の着信に開く封筒は\n', '踊り叱る赤い絵文字が\n', 'あの日隠し事したのばれたから\n', '謝り方考えなきゃな\n', 'いつもの僕らを\n', '見ればまあまあ平和なんだろう\n', '可愛げのある危機と\n', 'そうじゃない方胸に抱いて\n', '愛してるよ君を\n', '探してるよいつも\n', '他人のようで違う\n', '2人の折り合いを\n', '愛してるよ君を\n', '探してるよいつも\n', '家族のように映る\n', '2人の折り合いを\n', 'こんな時さえも誕生日は来て\n', '祝うだろう日々は続けと\n', 'うん、ごめんなさい \n', 'から始まる\n', 'バカ極まる毎度のことだが\n', '慣れてるわけじゃない\n', '藁をつかめテレビとかね\n', '気分変えるのに最適だ\n', 'えー\n', '多分漏れずクソみたいな話題\n', '観るくらいなら君と喧嘩していたい\n', '神へアーメン君への謝罪\n', 'まだ隠れラーメン食べたのしょない\n', 'どこへ行くの僕ら\n', 'いがみ合うよ彼ら\n', '空は晴れて風は\n', '髪を撫でていくわ\n', '家の中で僕ら\n', '道の上で僕ら\n', '空は晴れて風は\n', '心撫でていくわ\n', '愛してるよ君を\n', '探してるよいつも\n', '他人のようで違う\n', '2人の折り合いを\n', '愛してるよ君を\n', '探してるよいつも\n', '家族のように映る\n', '2人の折り合いを\n', 'たまに重なり合うよな僕ら\n', '扉閉じれば明日が生まれるなら\n', '遊ぼう一緒に\n', 'うちで踊ろう\n', 'ひとり踊ろう\n', '変わらぬ鼓動弾ませろよ\n', '生きて踊ろう\n', '僕らそれぞれの場所で\n', '重なり合うよ\n', 'うちで歌おう\n', '悲しみの向こう\n', '全ての歌で手を繋ごう\n', '生きてまた会おう\n', '僕らそれぞれの場所で\n', '重なり合えそう\n', 'うちで踊ろう\n', 'ひとり踊ろう\n', '変わらぬ鼓動弾ませろよ\n', '生きて踊ろう\n', '僕らそれぞれの場所で\n', '重なり合うよ\n', 'うちで歌おう\n', '悲しみの向こう\n', '全